# 클러스터에서 스파크 실행하기
- 이 장에서 알아볼 주제
  - 스파크 애플리케이션의 아키텍처와 컴포넌트
  - 스파크 내/외부에서 실행되는 스파크 애플리케이션의 생애주기
  - 파이프라이닝과 같은 중요한 저수준 실행 속성
  - 스파크 애플리케이션을 실행하는 데 필요한 사항

## 스파크 애플리케이션의 아키텍처
### 스파크 드라이버
  - 스파크 애플리케이션의 <strong>운전자 역할</strong>을 하는 프로세스
  - 스파크 애플리케이션의 <strong>실행을 제어</strong>하고
  - 스파크 클러스터(익스큐터의 상태와 태스크)의 <strong>모든 상태 정보를 유지</strong>하고
  - 물리적 컴퓨팅 자원 확보와 익스큐터 실행을 위해 <strong>클러스터 매니저와 통신</strong>할 수 있어야함

<br>

### 스파크 익스큐터
  - 스파크 익스큐터는 <strong>스파크 드라이버가 할당한 태스크를 수행</strong>하는 프로세스
  - 드라이버가 할당한 <strong>태스크를 받아 실행</strong>하고
  - 태스크의 <strong>상태와 결과(성공 또는 실패)를 드라이버에 보고</strong>함
  - 모든 스파크 애플리케이션은 <strong>개별 익스큐터 프로세스</strong>를 사용함

<br>

### 클러스터 매니저
  - 스파크 드라이버와 익스큐터를 허공에 띄울 수는 없으므로 클러스터 매니저가 필요
  - 클러스터 매니저는 <strong>스파크 애플리케이션을 실행할 클러스터 머신을 유지</strong>함
  - <strong>드라이버(마스터)와 워커</strong>라는 개념을 가지고 있음
  
  <img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FdTrTFM%2FbtqMgtI2j2i%2FEYx4YKL4F65mq2a0Yyx1LK%2Fimg.png" width=50%/>
  
    - 원은 개별 워커 노드를 실행하고 관리하는 데몬 프로세스
    - 위에 표시된 원들은 클러스터 매니저의 프로세스일 뿐
    
    
  - 스파크 애플리케이션을 실행할 때가 되면 클러스터 매니저에 자원 할당을 요청함
  - 스파크가 지원하는 클러스터 매니저
    - 스탠드얼론 클러스터 매니저
    - 아파치 메소스
    - 하둡 YARN

### 실행 모드
- 실행 모드는 애플리케이션을 실행할 때 요청한 자원의 물리적인 위치를 결정함
- 선택 가능한 실행 모드
  - 클러스터 모드
  - 클라이언트 모드
  - 로컬 모드

#### 클러스터 모드
- 가장 흔한 모드
- 클러스터 모드를 사용하려면 컴파일된 JAR파일이나 파이썬 스크립트 도는 R스크립트를 클러스터 매니저에 전달
- 클러스터 매니저는 파일은 받은 다음 <strong>워커 노드에 드라이버와 익스큐터 프로세스를 실행함</strong>
- 클러스터 매니저는 모든 스파크 애플리케이션과 관련된 <strong>프로세스를 유지</strong>하는 역할을 함



<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbJvmnR%2FbtqMmqdrL85%2FmlJL4vMXuDc0G3RGnWktwK%2Fimg.png" width=30% />
 
  - 위 그림은 하나의 워커 노드에 스파크 드라이버를 할당하고 다른 워커 노드에 익스큐터를 할당한 모습
  - 실선으로 그려진 직사각형: 드라이버 프로세스
  - 점선으로 그려진 직사각형: 익스큐터 프로세스

#### 클라이언트 모드
- 애플리케이션을 제출한 <strong>클라이언트 머신에 스파크 드라이버가 위치</strong>한다는 것을 제외하면 클러스터 모드와 비슷
  - 스파크 드라이버 프로세스를 유지하며 클러스터 매니저는 익스큐터 프로세스를 유지함
- 스파크 애플리케이션이 클러스터와 무관한 머신에서 동작
  - 보통 이런 머신을 게이트웨이 머신 또는 에지 노드라고 부름
  
  
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fb4iKEG%2FbtqMdDF3rDQ%2Fccqz2Y5UYwljpkKeaMDfXk%2Fimg.png" width=30% />
  
  
  - <strong>드라이버는 클러스터 외부의 머신에서 실행</strong>되고
  - 나머지 워커는 클러스터에 위치
  - 실선으로 그려진 직사각형: 드라이버 프로세스
  - 점선으로 그려진 직사각형: 익스큐터 프로세스

#### 로컬 모드
- 학습, 테스트 용도로 주로 사용
- 로컬 모드로 설정된 경우 모든 스파크 애플리케이션은 단일 머신에서 실행됨
- 로컬 모드는 애플리케이션의 병렬 처리를 위해 <strong>단일 머신의 스레드</strong>를 활용

## 스파크 애플리케이션의 생애주기(스파크 외부)
- 초기화부터 종료까지 스파크 애플리케이션의 생애주기를 단계별로 볼 것임

### 1단계: 클라이언트 요청

<img src ="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FIn7jj%2FbtqMdVNdQ8R%2FVfs1WRjog7iO7NIaYssKTk%2Fimg.png" width =30% />

<br>

  - 두꺼운 화살표 선: 스파크나 스파크 관련 프로세스가 수행하는 통신
  - 실선: 클러스터 매니저와의 통신 같은 일반적인 통신


#### 1. 스파크 애플리케이션(컴파일된 JAR나 라이브러리 파일) 제출
  - 제출하는 시점에 로컬 머신에서 코드가 실행되어 클러스터 드라이버 노드에 요청함
  - 이 과정에서 스파크 드라이버 프로세스의 자원을 함께 요청

#### 2. 클러스터 매니저는 이 요청을 받아들이고 클러스터 노드 중 하나에 드라이버 프로세스를 실행
#### 3. 스파크 잡을 제출한 클라이언트 프로세스는 종료되고 애플리케이션은 클러스터에서 실행

### 2단계: 시작

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbCCLbp%2FbtqMknad0vw%2Fn1tjDxHakavvfmZUdtmxy0%2Fimg.png" width=30% />

<br>

- 사용자 코드에는 <strong>반드시 스파크 클러스터(드라이버와 익스큐터)를 초기화하는 SparkSession이 포함</strong>되어야 함
  - SparkSession은 <strong>클러스터 매니저와 통신하여 스파크 익스큐터 프로세스의 실행을 요청</strong>함
  - 사용자는 spark-submit을 실행할 때 사용하는 명령행 인수로 익스큐터 수와 설정값을 지정 가능
  
#### 1. 클러스터 매니저는 익스큐터 프로세스를 시작하고 
#### 2. 결과를 응답 받아 익스큐터의 위치와 관련된 정보를 드라이버 프로세스로 전송
#### 3. 모든 작업이 정상적으로 완료되면 '스파크 클러스터'가 완성

### 3단계: 실행

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbSMFvs%2FbtqMgtWyyzw%2FiDqCWVEspmLMl0Unopfajk%2Fimg.png" width=30% />

<br>

#### 1. 드라이버와 워커는 코드를 실행하고 데이터를 이동하는 과정에서 서로 통신
#### 2. 드라이버는 각 워커에 태스크 할당
#### 3. 태스크를 할당받은 워커는 태스크의 상태와 성공/실패 여부를 드라이버에 전송

### 완료

#### 1. 스파크 애플리케이션의 실행이 완료되면 드라이버 프로세스가 성공이나 실패 중 하나의 상태로 종료됨
#### 2. 그 다음 클러스터 매니저는 드라이버가 속한 스파크 클러스터의 모든 익스큐터를 종료시킴
#### 3. 이 시점에 스파크 애플리케이션의 성공/실패 여부를 클러스터 매니저에 요청해 확인 가능